In [1]:
import requests
import numpy as np

In [2]:
d = [{'x': ['12/70 "Poleva-3" Slug', '12/70 "Poleva-6u" Slug',
       '12x70 Led slug', '12/70 Dual Sabot Slug',
       '12/70 FTX Custom LIte Slug', '12/70 HP Slug Copper Sabot Premier',
       '12/70 AP-20 Slug', '12/70 Grizzly 40 Slug',
       '12x70 shell with .50 BMG bullet', '12/70 HP Slug "SuperFormance"',
       '12x70 RIP']}]

In [3]:
for v in d[0]['x']:
    print(v.replace('12/70',''))

"Poleva-3" Slug
 "Poleva-6u" Slug
12x70 Led slug
 Dual Sabot Slug
 FTX Custom LIte Slug
 HP Slug Copper Sabot Premier
 AP-20 Slug
 Grizzly 40 Slug
12x70 shell with .50 BMG bullet
 HP Slug "SuperFormance"
12x70 RIP


## Web Scrape

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import html5lib

In [5]:
r = requests.get('https://escapefromtarkov.gamepedia.com/Ballistics#Armor_Penetration_Tables')

In [6]:
soup = BeautifulSoup(r.content,'html.parser')

In [7]:
table = soup.find_all("table",{"class":"wikitable sortable"})

In [8]:
df = pd.read_html(str(table),header=1)[1]

In [10]:
gun_dict = {'7.62x25mm Tokarev':['TT pistol','TT pistol (gold)'],
           '9x18mm Makarov': ['APB','APS','PB pistol','PM (t) pistol','PM pistol','PP-9 "Klin"',
           'PP-91 "Kedr"','PP-91-01 "Kedr-B"'],
           '9x19mm Parabellum':['GLOCK17','GLOCK18C','M9A3','MP-443 "Grach"','P226R',
                               'MP5','MP5K-N','MPX','PP-19-01 Vityaz-SN','Saiga-9','MP9','MP9-N'],
           '9x21mm Gyurza':['SR-1MP Gyurza'],
           '4.6x30mm HK':['MP7A1','MP7A2'],
           '5.7x28mm FN':['FN 5-7','P90'],
           '5.45x39mm':['AK-105','AK-74','AK-74M','AK-74N','AKS-74','AKS-74N','AKS-74U',
                       'AKS-74UB','AKS-74UN','RPK-16'],
           '5.56x45mm NATO':['ADAR 2-15','AK-101','AK-102','DT MDR','HK 416A5','M4A1','TX-15 DML'],
           '7.62x39mm':['OP-SKS','SKS','AK-103','AK-104','AKM','AKMN','AKMS','AKMSN','Vepr KM/VPO-136'],
           '7.62x51mm NATO':['Vepr Hunter/VPO-101','SA-58','DT MDR .308','M1A','RSASS','SR-25','DVL-10',
                            'M700','T-5000'],
           '7.62x54mmR':['SVDS','Mosin','Mosin Inf.','SV-98'],
           '9x39mm':['AS VAL','VSS Vintorez'],
           '.366 TKM':['Vepr AKM/VPO-209','VPO-215'],
           '12.7x55mm STs-130':['ASh-12'],
           '12.7x108mm':['NSV "Utes"'],
           '12/70 slugs':['M870','MP-133','MP-153','Saiga-12'],
           '20x70mm':['TOZ-106']}

## Cleaning

In [12]:
## need to remove tracer T and subsonic S at end of namesa

In [13]:
df.head()

,Caliber,Name,Fleshdamage,Penetrationpower,Armordamage %,Accuracy%,Recoil%,Fragmentationchance*,1,2,3,4,5,6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12/70 shot,12/70 5.25mm Buckshot**,8x34,1.0,15.0,NaN,NaN,0%,3.0,3.0,3.0,3.0,3.0,3.0
2,12/70 shot,"12/70 8.5 mm ""Magnum"" Buckshot**",8x44,2.0,26.0,-15.0,115.0,0%,3.0,3.0,3.0,3.0,3.0,3.0
3,12/70 shot,"12x70 6.5 mm ""Express"" Buckshot**",9x29,3.0,26.0,15.0,NaN,0%,3.0,3.0,3.0,3.0,3.0,3.0
4,12/70 shot,12x70 7mm Buckshot**,8x32,3.0,26.0,NaN,NaN,0%,3.0,3.0,3.0,3.0,3.0,3.0


In [14]:
df.drop(index=0,inplace=True)

In [15]:
df['Name'] = df['Name'].fillna(0).apply(lambda x: x.replace('*',''))
df.fillna(0,inplace=True)

In [16]:
new_names = []
for name in df['Name']:
    if (name[-4:] == ' T S'):
        df.loc[df['Name'] == name,'Name'] = name[:-4]
        new_names.append(name[:-4])
    if (name[-2:] == ' S'):
        df.loc[df['Name'] == name,'Name'] = name[:-2]
        new_names.append(name[:-2])
    if (name[-2:] == ' T'):
        df.loc[df['Name'] == name,'Name'] = name[:-2]
        new_names.append(name[:-2])


In [17]:
df.head()

,Caliber,Name,Fleshdamage,Penetrationpower,Armordamage %,Accuracy%,Recoil%,Fragmentationchance*,1,2,3,4,5,6
1,12/70 shot,12/70 5.25mm Buckshot,8x34,1.0,15.0,0.0,0.0,0%,3.0,3.0,3.0,3.0,3.0,3.0
2,12/70 shot,"12/70 8.5 mm ""Magnum"" Buckshot",8x44,2.0,26.0,-15.0,115.0,0%,3.0,3.0,3.0,3.0,3.0,3.0
3,12/70 shot,"12x70 6.5 mm ""Express"" Buckshot",9x29,3.0,26.0,15.0,0.0,0%,3.0,3.0,3.0,3.0,3.0,3.0
4,12/70 shot,12x70 7mm Buckshot,8x32,3.0,26.0,0.0,0.0,0%,3.0,3.0,3.0,3.0,3.0,3.0
5,12/70 shot,12/70 Flechette,8x19,28.0,26.0,-25.0,0.0,0%,6.0,6.0,6.0,5.0,5.0,4.0


In [18]:
df['Caliber'].value_counts()

9x18mm Makarov        14
5.45x39mm             12
12/70 slugs           11
5.56x45mm NATO         9
20x70mm                8
7.62x25mm Tokarev      7
5.7x28mm FN            7
7.62x54mmR             6
7.62x51mm NATO         6
9x19mm Parabellum      6
7.62x39mm              5
12/70 shot             5
9x39mm                 4
4.6x30mm HK            4
9x21mm Gyurza          4
.366 TKM               3
Stationary Weapons     3
12.7x55mm STs-130      3
.45 ACP                2
Name: Caliber, dtype: int64

In [19]:
df['price'] = 0
df['diff24h']= 0
df['diff7days'] = 0
df['updated'] = 0

## Get Pricing Data

In [20]:
# need to change mm to match API
df.loc[df['Name'] == '5.56x45 mm Warmage','Name'] = '5.56x45mm Warmage'

In [21]:
api_key = 'nTzwgv4kSJLcD3yS'
url = 'https://tarkov-market.ru/api/v1/'

In [22]:
headers = {
    'x-api-key': 'nTzwgv4kSJLcD3yS',
}

In [23]:
params = (('q', '12/70 5.25mm Buckshot'),)
response = requests.get('https://tarkov-market.com/api/v1/item', headers=headers, params=params)

In [24]:
response.text

'[{"uid":"92eb900d-4a41-429c-b062-db19dcb3012c","name":"12/70 5.25mm Buckshot","shortName":"5.25 12c","price":36,"avg24hPrice":36,"avg7daysPrice":170,"traderName":"","traderPrice":0,"traderPriceCur":"₽","updated":"2020-07-02T19:35:10.522Z","slots":1,"diff24h":0,"diff7days":-78.82,"icon":"https://gamepedia.cursecdn.com/escapefromtarkov_gamepedia/4/4e/1270525BuckshotIcon.png?version=b59d78ed4a323c3c1ce8b4439b475d30","link":"https://tarkov-market.com/item/12_70_5.25mm_buckshot","wikiLink":"https://escapefromtarkov.gamepedia.com/12/70_5.25mm_Buckshot","img":"https://gamepedia.cursecdn.com/escapefromtarkov_gamepedia/4/4e/1270525BuckshotIcon.png?version=b59d78ed4a323c3c1ce8b4439b475d30","imgBig":"https://gamepedia.cursecdn.com/escapefromtarkov_gamepedia/thumb/c/c4/12x70BUCKSHOTIMAGE.png/320px-12x70BUCKSHOTIMAGE.png?version=903b276d0036bea2608f2ea5a918d9e2","bsgId":"5d6e6772a4b936088465b17c","reference":"https://www.patreon.com/tarkov_market"}]'

In [25]:
missing_names = []
unclear_name = []
for name in df['Name']:

    params = (('q', name),)
    response = requests.get('https://tarkov-market.com/api/v1/item', headers=headers, params=params)
    res = response.json()
    if len(res) >= 1:
        if res[0]['name'] == name:
            df.loc[df['Name'] == name,'price'] = res[0]['price']
            df.loc[df['Name'] == name,'diff24h'] = res[0]['diff24h']
            df.loc[df['Name'] == name,'diff7days'] = res[0]['diff7days']
            df.loc[df['Name'] == name,'updated'] = res[0]['updated']
        else:
            unclear_name.append(name)
    else:
        missing_names.append(name)
    response.close()

In [26]:
df['Caliber'].unique()

array(['12/70 shot', '12/70 slugs', '20x70mm', '9x18mm Makarov',
       '7.62x25mm Tokarev', '9x19mm Parabellum', '.45 ACP',
       '9x21mm Gyurza', '5.7x28mm FN', '4.6x30mm HK', '9x39mm',
       '.366 TKM', '5.45x39mm', '5.56x45mm NATO', '7.62x39mm',
       '7.62x51mm NATO', '7.62x54mmR', '12.7x55mm STs-130',
       'Stationary Weapons'], dtype=object)

In [27]:
df.columns

Index(['Caliber', 'Name', 'Fleshdamage', 'Penetrationpower', 'Armordamage %',
       'Accuracy%', 'Recoil%', 'Fragmentationchance*', '1', '2', '3', '4', '5',
       '6', 'price', 'diff24h', 'diff7days', 'updated'],
      dtype='object')

In [28]:
cols_rename = {'Caliber':'Caliber',
                'Name':'Bullet Name',
              'Fleshdamage':'Flesh Damage',
              'Penetrationpower':'Penetration Power',
              'Armordamage\xa0%':'Armor Damage (%)',
              'Accuracy%':'Accuracy (%)',
              'Recoil%':'Recoil (%)',
              'Fragmentationchance*': 'Fragmentation Chance (%)',
              '1':'Pentration Armor One',
              '2':'Pentration Armor Two',
              '3':'Pentration Armor Three',
              '4':'Pentration Armor Four',
              '5':'Pentration Armor Five',
              '6':'Pentration Armor Six',
              'price':'Current Price',
              'diff24h':'24 hour Price Diff',
              'diff7days':'1 week Price Diff',
              'updated':'Last Updated'}

In [29]:
df.rename(columns=cols_rename,inplace=True)

In [30]:
df.rename(columns={'Armordamage\xa0%':'Armor Damage (%)'},inplace=True)

In [31]:
# convert 8x19 to total pellet damage
flesh_d = df.loc[df['Flesh Damage'].str.contains('x')]['Flesh Damage']

In [32]:
total_flesh = []
for item in flesh_d:
    total_flesh.append(int(item[0]) * int(item[2:]))

In [33]:
df.loc[df['Flesh Damage'].str.contains('x'), 'Flesh Damage'] = total_flesh

In [34]:
df['Flesh Damage'] = pd.to_numeric(df['Flesh Damage'])

In [35]:
df.head()

,Caliber,Bullet Name,Flesh Damage,Penetration Power,Armor Damage (%),Accuracy (%),Recoil (%),Fragmentation Chance (%),Pentration Armor One,Pentration Armor Two,Pentration Armor Three,Pentration Armor Four,Pentration Armor Five,Pentration Armor Six,Current Price,24 hour Price Diff,1 week Price Diff,Last Updated
1,12/70 shot,12/70 5.25mm Buckshot,272,1.0,15.0,0.0,0.0,0%,3.0,3.0,3.0,3.0,3.0,3.0,36,0.00,-78.82,2020-07-02T19:35:10.522Z
2,12/70 shot,"12/70 8.5 mm ""Magnum"" Buckshot",352,2.0,26.0,-15.0,115.0,0%,3.0,3.0,3.0,3.0,3.0,3.0,172,-6.01,-10.88,2020-07-02T19:24:51.815Z
3,12/70 shot,"12x70 6.5 mm ""Express"" Buckshot",261,3.0,26.0,15.0,0.0,0%,3.0,3.0,3.0,3.0,3.0,3.0,50,2.04,-69.14,2020-07-02T16:45:51.245Z
4,12/70 shot,12x70 7mm Buckshot,256,3.0,26.0,0.0,0.0,0%,3.0,3.0,3.0,3.0,3.0,3.0,24,-17.24,-14.29,2020-07-02T16:13:19.750Z
5,12/70 shot,12/70 Flechette,152,28.0,26.0,-25.0,0.0,0%,6.0,6.0,6.0,5.0,5.0,4.0,2000,33.33,77.46,2020-07-02T14:06:50.634Z


In [36]:
df['Pen Power by Price'] = df['Penetration Power'] / df['Current Price']
df['Flesh Damage by Price'] = df['Flesh Damage'] / df['Current Price']

In [37]:
df.replace(np.inf,0,inplace=True)

## Cleaning New DF

In [38]:
df.to_csv('init.csv')

In [39]:
df.head()

,Caliber,Bullet Name,Flesh Damage,Penetration Power,Armor Damage (%),Accuracy (%),Recoil (%),Fragmentation Chance (%),Pentration Armor One,Pentration Armor Two,Pentration Armor Three,Pentration Armor Four,Pentration Armor Five,Pentration Armor Six,Current Price,24 hour Price Diff,1 week Price Diff,Last Updated,Pen Power by Price,Flesh Damage by Price
1,12/70 shot,12/70 5.25mm Buckshot,272,1.0,15.0,0.0,0.0,0%,3.0,3.0,3.0,3.0,3.0,3.0,36,0.00,-78.82,2020-07-02T19:35:10.522Z,0.027778,7.555556
2,12/70 shot,"12/70 8.5 mm ""Magnum"" Buckshot",352,2.0,26.0,-15.0,115.0,0%,3.0,3.0,3.0,3.0,3.0,3.0,172,-6.01,-10.88,2020-07-02T19:24:51.815Z,0.011628,2.046512
3,12/70 shot,"12x70 6.5 mm ""Express"" Buckshot",261,3.0,26.0,15.0,0.0,0%,3.0,3.0,3.0,3.0,3.0,3.0,50,2.04,-69.14,2020-07-02T16:45:51.245Z,0.060000,5.220000
4,12/70 shot,12x70 7mm Buckshot,256,3.0,26.0,0.0,0.0,0%,3.0,3.0,3.0,3.0,3.0,3.0,24,-17.24,-14.29,2020-07-02T16:13:19.750Z,0.125000,10.666667
5,12/70 shot,12/70 Flechette,152,28.0,26.0,-25.0,0.0,0%,6.0,6.0,6.0,5.0,5.0,4.0,2000,33.33,77.46,2020-07-02T14:06:50.634Z,0.014000,0.076000


In [40]:
df[['Current Price','Bullet Name']].sort_values(by='Current Price').iloc[-1]['Bullet Name']

'12/70 Flechette'

In [ ]:
t = df.loc[df['Bullet Name'] == '12/70 Grizzly 40 Slug']

In [ ]:
t.loc['Pentration Armor One':'Pentration Armor Six']

In [ ]:
df.loc[df['Bullet Name'] == '12/70 Grizzly 40 Slug'].loc[:,'Pentration Armor One':'Pentration Armor Six'].values